In [46]:
import pandas as pd
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
from datetime import datetime
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import ParameterGrid
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

def perform_grid_search(training_pairs, cv_pairs):
    """Perform grid search across all training pairs."""
    param_grid = {
        'max_depth': [1, 2, 4],
        'n_estimators': [25, 50, 100, 400],
        'learning_rate': [0.005, 0.01, 0.2],
    }

    best_params = None
    best_score = float('inf')

    for params in ParameterGrid(param_grid):
        existing_model = None
        
        for i in range(len(training_pairs)):
            existing_model = train_and_evaluate(training_pairs[i][0], cv_pairs, training_pairs[i][1], existing_model, True)
        
        total_cv_score = 0
        
        for cv_X, cv_y in cv_pairs:
            cv_pred = existing_model.predict(cv_X)
            total_cv_score += mean_squared_error(cv_pred, cv_y)
            
        total_cv_score /= len(cv_pairs)
        
        if total_cv_score < best_score:
            best_score = total_cv_score
            best_params = params

    return best_params

def train_and_evaluate(X_train, CV_grids, y_train, existing_model, grid_search = False):
     
    if(existing_model == None):
        existing_model = XGBRegressor(n_estimators=1000, learning_rate=0.01, max_depth=4)
    
    existing_model.fit(X_train, y_train, verbose = True)
    
    if not grid_search:
        y_train_pred = existing_model.predict(X_train)    
        train_mse = mean_squared_error(y_train, y_train_pred)    
        train_mae = mean_absolute_error(y_train, y_train_pred)
        train_rmse = mean_squared_error(y_train, y_train_pred, squared=False)

        cv_mse = cv_mae = cv_rmse = 0
        n = len(CV_grids)

        for index, pair in enumerate(CV_grids):
            weather, burn = pair
            cv_pred = existing_model.predict(weather)
            cv_mse += mean_squared_error(cv_pred, burn)
            cv_rmse += mean_squared_error(cv_pred, burn, squared = False)
            cv_mae += mean_absolute_error(cv_pred, burn)

        cv_mse /= n
        cv_rmse /= n
        cv_mae /= n

        print('Train Mean Squared Error:', train_mse)
        print('CV Mean Squared Error:', cv_mse)
    
    if grid_search:
        return existing_model
    else:
        return existing_model, train_mse, cv_mse, train_mae, cv_mae, train_rmse, cv_rmse

def save_model(model, year, save_dir = '../models'):
    # Saving the XGBoost model
    os.makedirs(save_dir, exist_ok=True)
    model_file_name = f'xgb_model_year_{year}.bin'
    model.save_model(os.path.join(save_dir, model_file_name))
    print(f'Model saved to {model_file_name}')
    
def get_training_data():
    training_data = []
    for group in range(1, 25):
        weather_path = '../data/prepared/quarterly-6-by-6/weather/group_' + str(group) + '.csv'
        burn_path = '../data/prepared/quarterly-6-by-6/satellite-burn/group_' + str(group + 1) + '.csv'
        
        weather_data_df = pd.read_csv(weather_path)
        burn_data = np.genfromtxt(burn_path, delimiter=',')

        X = scaler.fit_transform(weather_data_df.values)
        y = burn_data

        train_pair = (X, y)
        training_data.append(train_pair)

    return training_data
    
def xgboost_trainer():
    model = None

    cv_mae_array = []
    train_mae_array = []
    
    cv_rmse_array = []
    train_rmse_array = []
    
    cv_mse_array = []
    train_mse_array = []
            
    existing_model = None
        
    training_data = get_training_data()
    
    test_pairs = []
    
    cv_pairs = []
    
    train_pairs = []
    
    for group, pair in enumerate(training_data):
        weather_data, burn_data = pair
        
        if group in [4,9,14,19]:
            test_pairs.append((weather_data, burn_data))
            continue
        elif group in [3,7,11,15,20]:
            cv_pairs.append((weather_data, burn_data))
            continue
        else:
            train_pairs.append((weather_data, burn_data))
            
    best_params = perform_grid_search(train_pairs, cv_pairs) # grid search to optimize max depth, n estimators and learning rate for XGBoost model, resulting in notable improvements in loss
    
    print(best_params)
    
    existing_model = XGBRegressor(**best_params)
    
    for group, pair in enumerate(train_pairs):
        weather, burn = pair
        existing_model, cv_mse, train_mse, cv_mae, train_mae, cv_rmse, train_rmse = train_and_evaluate(weather, cv_pairs, burn, existing_model)

        cv_mae_array.append(cv_mae)
        train_mae_array.append(train_mae)
        cv_mse_array.append(cv_mse)
        train_mse_array.append(train_mse)
        cv_rmse_array.append(cv_rmse)
        train_rmse_array.append(train_rmse)
        
    save_results(existing_model, train_mae_array, cv_mae_array, train_rmse_array, cv_rmse_array, train_mse_array, cv_mse_array)
    
    final_test_results(existing_model, test_pairs)
    
def final_test_results(model, test_data):
    n = len(test_data)
    mse = 0
    mae = 0
    rmse = 0
    
    for (X, y) in test_data:
        y_pred = model.predict(X)
    
        mse += mean_squared_error(y, y_pred)
        rmse += mean_squared_error(y, y_pred, squared=False)
        mae += mean_absolute_error(y, y_pred)
    
    print(mse / n, mae / n, rmse / n)
        

def get_run_notes():
    time_step = "time-step-"
    time_step += input("Enter the time step: ")
    fine_areas = "fine-areas-"
    fine_areas += input("Enter the number of fine areas: ")
    return time_step, fine_areas

def save_results(existing_model, train_mae_array, cv_mae_array, train_rmse_array, cv_rmse_array, train_mse_array, cv_mse_array):
    # Get information about this run 
    os.makedirs('../models', exist_ok = True)
    model_file_name = f'xgb_model_{4}'
    existing_model.save_model('../models/' + model_file_name)
    
    time_step, fine_areas = get_run_notes()

    # Get the current time
    current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

    # Create a folder with the current time as the name
    base_folder_name = "../evaluation"
    folder_name = f"{base_folder_name}/{current_time}_{time_step}_{fine_areas}"
    try:
        os.mkdir(folder_name)
        print(f"Folder '{folder_name}' created successfully.")
    except FileExistsError:
        print(f"Folder '{folder_name}' already exists.")

    # Mean Absolute Errors Plot
    plt.figure()
    sns.lineplot(x=range(len(train_mae_array)), y=train_mae_array, marker='o', label='Training')
    sns.lineplot(x=range(len(cv_mae_array)), y=cv_mae_array, marker='o', label='Cross-Validation')
    plt.xlabel('Index')
    plt.ylabel('Mean Absolute Error')
    plt.title('Mean Absolute Errors Plot')
    plt.legend()
    plt.savefig(f'{folder_name}/mean_absolute_errors.pdf')
    plt.close()

    # Mean Squared Errors Plot
    plt.figure()
    sns.lineplot(x=range(len(train_mse_array)), y=train_mse_array, marker='o', label='Training')
    sns.lineplot(x=range(len(cv_mse_array)), y=cv_mse_array, marker='o', label='Cross-Validation')
    plt.xlabel('Index')
    plt.ylabel('Mean Squared Error')
    plt.title('Mean Squared Errors Plot')
    plt.legend()
    plt.savefig(f'{folder_name}/mean_squared_errors.pdf')
    plt.close()

    # Root Mean Squared Errors Plot
    plt.figure()
    sns.lineplot(x=range(len(train_rmse_array)), y=train_rmse_array, marker='o', label='Training')
    sns.lineplot(x=range(len(cv_rmse_array)), y=cv_rmse_array, marker='o', label='Cross-Validation')
    plt.xlabel('Index')
    plt.ylabel('Root Mean Squared Error')
    plt.title('Root Mean Squared Errors Plot')
    plt.legend()
    plt.savefig(f'{folder_name}/root_mean_squared_errors.pdf')
    plt.close()

In [ ]:
xgboost_trainer()

{'learning_rate': 0.005, 'max_depth': 1, 'n_estimators': 25}
Train Mean Squared Error: 0.6930457997867688
CV Mean Squared Error: 2995.744626851302
Train Mean Squared Error: 556078.6322872441
CV Mean Squared Error: 41680.01080845989
Train Mean Squared Error: 0.9504414729618565
CV Mean Squared Error: 2997.3346556951265
Train Mean Squared Error: 1012855.4454277764
CV Mean Squared Error: 209868.45057169147
Train Mean Squared Error: 0.0
CV Mean Squared Error: 2999.394380555556
Train Mean Squared Error: 2907.302616762826
CV Mean Squared Error: 3029.426819561089
Train Mean Squared Error: 0.0
CV Mean Squared Error: 2999.394380555556
Train Mean Squared Error: 10.485557928352423
CV Mean Squared Error: 2985.1828768580826
Train Mean Squared Error: 199.5151932285986
CV Mean Squared Error: 2961.171549142896


/Users/aryans0921/opt/anaconda3/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [00:37:22] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)


Train Mean Squared Error: 173640.8068172059
CV Mean Squared Error: 12715.40061809557
Train Mean Squared Error: 728279277.8993003
CV Mean Squared Error: 34441580.3324373
Train Mean Squared Error: 1.009640952928121
CV Mean Squared Error: 2997.4249957810416
Train Mean Squared Error: 10696.691126887617
CV Mean Squared Error: 3202.7199250431045
Train Mean Squared Error: 40.06049445240436
CV Mean Squared Error: 2983.35213329658
Train Mean Squared Error: 109.08534585210117
CV Mean Squared Error: 2981.5351776582693


In [40]:
loaded_model = XGBRegressor()

In [41]:
loaded_model.load_model('../models/xgb_model_2')

In [42]:
training_data = get_training_data()
    
test_pairs = []

cv_pairs = []

train_pairs = []

for group, pair in enumerate(training_data):
    weather_data, burn_data = pair

    if group in [4,9,14,19]:
        test_pairs.append((weather_data, burn_data))
        continue
    elif group in [3,7,11,15,20]:
        cv_pairs.append((weather_data, burn_data))
        continue
    else:
        train_pairs.append((weather_data, burn_data))